<a href="https://colab.research.google.com/github/ChicagoPark/chicago_work/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



##    전체적 모델링 흐름

*   본 프로젝트 분류를 위해 여러 차례 훈련했던 MobileNetV2 에서 Convolution layer 을 Feature extractor 으로 활용하고, 이에 뒤 뉴럴네트워크 연산 부분의 노드 구성을 달리하여 10개의 모델을 준비한다.

*   "---" 를 컴퓨터 계산적으로 증명된 Hyper parameter 를 사용하기위해 Hyperparameter tuning 기법을 활용한다.

## 본 Colab 모델링 목차


1.   필요한 라이브러리 및 데이터 전처리
2.   feature extractor 모델을 가져와서, 특징 추출기 자리에 넣고 freezing 하기
3. 모델에 Opimizer, Loss 등을 명시하고, 훈련을 수행
4. TensorBoard 를 활용한 훈련결과 시각화
5. Test Set 에 대해 4X4 로 보여주는 결과

## 본 Colab 부록


1.   Hypertuning 진행
2.   항목 추가

Model 구조 : https://viscom.net2vis.uni-ulm.de/kDjvWN6kxJ8UBo6loBvuO0OoRQgKDSS5ndoGbdULXBSbbwdV7P



In [ ]:
%%capture
# Necessary Modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation(Image transformations) 을 하기 위해 필요한 모듈

import scipy
import keras
from tensorflow.keras.utils import plot_model
! pip install keras_tuner
import keras_tuner as kt
import shutil
#! pip install tensorflow-gpu

#! pip3 install --upgrade tensorflow-gpu==1.8.0

#!pip install -q gwpy # 너무 많이 출력되는 셀을 출력막기위해 

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Colab 상의 훈련용 데이터 디렉터리 정리
!mkdir training_data
%cd training_data/
!mkdir '0. stone'
!mkdir '1. pothole'
!mkdir '2. filled'
%cd ..

/content/training_data
/content


In [ ]:
%%capture
'''
# data path
# 인터넷에서 주워온것임 ! gdown --id 1PGhXUnaJDpcjgoLEhNn9gRvGZGRCzWUt
'''

# 빅데이터 AI 수업에서 공유된 data.zip 
#! gdown --id 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO --out 'data.zip'

# data.zip 학교제공 : 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO / 내 드라이브 : 1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA

# 빅데이터 AI 수업에서 공유된 실차 데이터.zip
#! gdown --id 12PwxkFYqIBiI_7Upxfyxg9Aq6MbmO2Wm --out '실차데이터.zip'

# 실차데이터 학교제공 : 1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L / 내 드라이브 : 12PwxkFYqIBiI_7Upxfyxg9Aq6MbmO2Wm

# json parsing 에서 나누었던 데이터.zip
#! gdown --id 1Dr8z1zXNpTZVqcKd1cYn0d71gi5itA3W --out 'divided_data.zip'

# 하이퍼파라미터 설정을 위한 tuner_data.zip 파일
#! gdown --id 1Fd9OcOzhzYwhmDS38bUTqdvoOwSOH8jc --out 'tuning_data.zip'

# Model 과 각종 기능이 잘 동작하는지 확인하기위한 간단한 파일
#! gdown --id 1NBl5yFuPBOtc59ikZJtx8_xPN-EuCcjt

# 직접 데이터 추출 및 작업을 진행한 구글드라이브 상 데이터
# ! gdown --id <추가예정> --out '<설정하고자하는폴더명>.zip'



! gdown --id 1pJMT9F__i0dGlpNnUabY8wwU5jPzS7Nw --out 'hopefully_main_data_with_Kaggle.zip'

!unzip 'hopefully_main_data_with_Kaggle.zip' -d 'hopefully_main_data_with_Kaggle'

#! gdown --id 18oGJxvLJ3HfQ4VJf40oBch6P9FKS9gr- --out 'hopefully_main_data.zip'

#!unzip 'hopefully_main_data.zip' -d 'hopefully_main_data'

'''
data.zip 파일은 큰 파일이니 아래 절차로 zip 해제할 것
! gdown --id 1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA --out 'data.zip'
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''

#!unzip 'tuning_data.zip' -d 'tuning_data'



#!unzip 'first_checking_data.zip' -d first_checking_data

'''
!unzip '실차데이터.zip' -d '실차데이터'
!unzip 'divided_data.zip' -d 'divided_data'

! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''

In [ ]:

# training_data 폴더로 zip 해제한 모든 파일을 정리한다.
'''
!mv /content/실차데이터/'0. no_pothole'/* /content/training_data/'0. stone'
!mv /content/실차데이터/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/실차데이터/'2. filled'/* /content/training_data/'2. filled'

!mv /content/divided_data/'0. stone/Images'/* /content/training_data/'0. stone'
!mv /content/divided_data/'1. pothole/Images'/* /content/training_data/'1. pothole'
!mv /content/divided_data/'2. filled/Images'/* /content/training_data/'2. filled'

!mv /content/data/data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/data/data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/data/data/'2. filled'/* /content/training_data/'2. filled'

!mv /content/tuning_data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/tuning_data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/tuning_data/'2. filled'/* /content/training_data/'2. filled'

!mv /content/first_checking_data/first_checking_data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/first_checking_data/first_checking_data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/first_checking_data/first_checking_data/'2. filled'/* /content/training_data/'2. filled'

!mv /content/hopefully_main_data/hopefully_main_data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/hopefully_main_data/hopefully_main_data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/hopefully_main_data/hopefully_main_data/'2. filled'/* /content/training_data/'2. filled'
'''
!mv /content/hopefully_main_data_with_Kaggle/hopefully_main_data_with_Kaggle/'0. stone'/* /content/training_data/'0. stone'
!mv /content/hopefully_main_data_with_Kaggle/hopefully_main_data_with_Kaggle/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/hopefully_main_data_with_Kaggle/hopefully_main_data_with_Kaggle/'2. filled'/* /content/training_data/'2. filled'

In [ ]:
# 1. 기본 뿌리 디렉터리와 검증데이터 경로를 가져온다
base_path = os.path.abspath('/content/training_data')
print(base_path)

# 2. 데이터 경로를 지정을 해준다.
no_pothole_dir = os.path.join(base_path, '0. stone')
pothole_dir = os.path.join(base_path, '1. pothole')
filled_dir = os.path.join(base_path, '2. filled')

# 3. 데이터 셋의 개수를 카운팅한다.
num_no_pothole = len(os.listdir(no_pothole_dir))
num_pothole_dir = len(os.listdir(pothole_dir))
num_filled_dir = len(os.listdir(filled_dir))

total_data = num_no_pothole + num_pothole_dir + num_filled_dir

print(f'정상도로 : {num_no_pothole} 개 , 팟홀도로 : {num_pothole_dir}, 보수도로 : {num_filled_dir}')

print(f'Total Data 개수 : {total_data}')

/content/training_data
정상도로 : 1554 개 , 팟홀도로 : 1832, 보수도로 : 1703
Total Data 개수 : 5089


In [ ]:
# Train set 과 Validation set 을 나누며, Data Augmentation 작업 또한 추가시킨다.
train_image_generator = ImageDataGenerator(rescale=1./255,                                     
      rotation_range=7,
      #width_shift_range=0.1,
      #height_shift_range=0.1,
      #shear_range=0.1,
      #zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.15)
'''

# without augmentation
train_image_generator = ImageDataGenerator(rescale=1./255,
      validation_split=0.15)
'''

validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                horizontal_flip=True,
                                                validation_split=0.15)
'''
batch_size = 32
epochs = 15
IMG_HEIGHT = 320
IMG_WIDTH = 320
'''
train_batch_size = 32
val_batch_size = 32

epochs = 10
IMG_HEIGHT = 320
IMG_WIDTH = 320

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=train_batch_size,
                                                           directory=base_path,
                                                           seed = 22,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='training',
                                                           class_mode='categorical')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=val_batch_size,
                                                           directory=base_path,
                                                           seed = 22,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='validation',
                                                           class_mode='categorical')
train_num = train_data_gen.samples # training set 이 몇개인지
val_num = val_data_gen.samples   # validation set 이 몇개인지

Found 4327 images belonging to 3 classes.
Found 762 images belonging to 3 classes.


# Feature Extractor 가져오기

In [ ]:
feature_extractor = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/빅데이터및AI/Model/model_11.h5')

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
base_model.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = base_model(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=320,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=128,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=96,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=32,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model = keras.Model(inputs=inputs, outputs=outputs)

new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 320)               409920    
                                                                 
 batch_normalization_5 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                           

In [ ]:
# 3번, 4번
# 내부 변수 확인 print(feature_extractor.layers[4].get_weights())


In [ ]:
# first index is about feature extractor
new_model.layers[1].set_weights(feature_extractor.layers[1].get_weights())
new_model.layers[3].set_weights(feature_extractor.layers[3].get_weights())
new_model.layers[4].set_weights(feature_extractor.layers[4].get_weights())
# 특징추출기에 해당하는 CNN 을 freezing 시킨다.
new_model.layers[1].trainable = False
new_model.layers[3].trainable = False
new_model.layers[4].trainable = False

new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 320)               409920    
                                                                 
 batch_normalization_5 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                           

In [ ]:
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, centered=True)
opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
new_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/빅데이터및AI/Model/model_12_11_12.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

In [ ]:
history = new_model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=15,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/15
135/135 [==============================] - ETA: 0s - loss: 0.1563 - accuracy: 0.9502

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


135/135 [==============================] - 456s 3s/step - loss: 0.1563 - accuracy: 0.9502 - val_loss: 1.6824 - val_accuracy: 0.6332
Epoch 2/15
135/135 [==============================] - 430s 3s/step - loss: 0.1302 - accuracy: 0.9560 - val_loss: 1.4894 - val_accuracy: 0.6658
Epoch 3/15
135/135 [==============================] - 410s 3s/step - loss: 0.1052 - accuracy: 0.9641 - val_loss: 1.5658 - val_accuracy: 0.6467
Epoch 4/15
135/135 [==============================] - 408s 3s/step - loss: 0.1026 - accuracy: 0.9639 - val_loss: 1.4740 - val_accuracy: 0.6603
Epoch 5/15
110/135 [=======================>......] - ETA: 1:09 - loss: 0.1134 - accuracy: 0.9605

In [ ]:
new_model.history.history

{'accuracy': [0.9110127687454224,
  0.9284660816192627,
  0.9454277157783508,
  0.9557521939277649,
  0.9601770043373108,
  0.9584562182426453,
  0.9461652040481567,
  0.9601770043373108,
  0.9653392434120178,
  0.9646017551422119,
  0.9643559455871582,
  0.9668141603469849,
  0.9690265655517578,
  0.9650934338569641,
  0.9591937065124512,
  0.9663225412368774,
  0.966076672077179,
  0.9756637215614319,
  0.9761553406715393,
  0.9736971259117126,
  0.9830383658409119,
  0.9594395160675049,
  0.9621435403823853,
  0.9692723751068115,
  0.9692723751068115,
  0.9746804237365723,
  0.9727138876914978,
  0.9594395160675049,
  0.9788593649864197,
  0.9788593649864197,
  0.9788593649864197,
  0.9695181846618652,
  0.9631268382072449,
  0.9594395160675049,
  0.9744346141815186,
  0.9783677458763123,
  0.9842674732208252,
  0.9845132827758789,
  0.9864798188209534,
  0.9218289256095886,
  0.9535398483276367,
  0.9658308625221252,
  0.9741888046264648,
  0.9736971259117126,
  0.9761553406715393,

In [ ]:
# decision boundary
# https://www.machinecurve.com/index.php/2019/10/11/how-to-visualize-the-decision-boundary-for-your-keras-model/

In [ ]:
base_model2 = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
base_model2.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = base_model2(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=1024,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=486,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=128,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=32,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model2 = keras.Model(inputs=inputs, outputs=outputs)

#new_model2.summary()

new_model2.layers[1].set_weights(feature_extractor.layers[1].get_weights())

new_model2.layers[1].trainable = False
new_model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 1024)              1311744   
                                                                 
 batch_normalization_4 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                           

In [ ]:
opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
new_model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])
my_callbacks2 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/빅데이터및AI/Model/model_12_using_freezing.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

In [ ]:
history2 = new_model2.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=10,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks2])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
136/136 [==============================] - ETA: 0s - loss: 0.1303 - accuracy: 0.9543

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


136/136 [==============================] - 337s 2s/step - loss: 0.1303 - accuracy: 0.9543 - val_loss: 1.2351 - val_accuracy: 0.8069
Epoch 2/10
136/136 [==============================] - 329s 2s/step - loss: 0.1114 - accuracy: 0.9604 - val_loss: 1.3497 - val_accuracy: 0.7764
Epoch 3/10
136/136 [==============================] - 330s 2s/step - loss: 0.0891 - accuracy: 0.9705 - val_loss: 1.8045 - val_accuracy: 0.7347
Epoch 4/10
136/136 [==============================] - 330s 2s/step - loss: 0.0883 - accuracy: 0.9693 - val_loss: 1.3624 - val_accuracy: 0.7847
Epoch 5/10
136/136 [==============================] - 332s 2s/step - loss: 0.0968 - accuracy: 0.9656 - val_loss: 1.5401 - val_accuracy: 0.7583
Epoch 6/10
136/136 [==============================] - 331s 2s/step - loss: 0.0799 - accuracy: 0.9717 - val_loss: 2.0351 - val_accuracy: 0.7542
Epoch 7/10
136/136 [==============================] - 331s 2s/step - loss: 0.0886 - accuracy: 0.9700 - val_loss: 1.1995 - val_accuracy: 0.7764
Epoch 8/10

In [ ]:
base_model3 = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
base_model3.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = base_model3(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=1024,activation=None)(x)
x = tf.keras.layers.GaussianDropout(0.4)(x)
x = tf.keras.layers.Activation(activation="relu")(x)
x = tf.keras.layers.Dense(units=512,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GaussianDropout(0.4)(x)
x = tf.keras.layers.Activation(activation="relu")(x)
x = tf.keras.layers.Dense(units=128,activation=None)(x)
x = tf.keras.layers.GaussianDropout(0.4)(x)
x = tf.keras.layers.Activation(activation="relu")(x)
x = tf.keras.layers.Dense(units=64,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GaussianDropout(0.4)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model3 = keras.Model(inputs=inputs, outputs=outputs)

#new_model2.summary()

new_model3.layers[1].set_weights(feature_extractor.layers[1].get_weights())

new_model3.layers[1].trainable = False
new_model3.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 1024)              1311744   
                                                                 
 gaussian_dropout (GaussianD  (None, 1024)             0         
 ropout)                                                         
                                                           

In [ ]:
opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
new_model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])
my_callbacks3 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/빅데이터및AI/Model/model_13.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

In [ ]:
history3 = new_model3.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=10,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks3])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 0.1723 - accuracy: 0.9454

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


128/128 [==============================] - 388s 3s/step - loss: 0.1723 - accuracy: 0.9454 - val_loss: 2.2066 - val_accuracy: 0.7472
Epoch 2/10
128/128 [==============================] - 360s 3s/step - loss: 0.1108 - accuracy: 0.9663 - val_loss: 1.5843 - val_accuracy: 0.7557
Epoch 3/10
128/128 [==============================] - 357s 3s/step - loss: 0.1071 - accuracy: 0.9680 - val_loss: 1.8381 - val_accuracy: 0.7330
Epoch 4/10
128/128 [==============================] - 356s 3s/step - loss: 0.1004 - accuracy: 0.9668 - val_loss: 1.2898 - val_accuracy: 0.7756
Epoch 5/10
128/128 [==============================] - 356s 3s/step - loss: 0.1058 - accuracy: 0.9663 - val_loss: 1.6187 - val_accuracy: 0.7500
Epoch 6/10
128/128 [==============================] - 356s 3s/step - loss: 0.0967 - accuracy: 0.9671 - val_loss: 1.4923 - val_accuracy: 0.7543
Epoch 7/10
128/128 [==============================] - 349s 3s/step - loss: 0.0920 - accuracy: 0.9720 - val_loss: 1.5405 - val_accuracy: 0.7571
Epoch 8/10

# Hyperparam Tuning : 0.35 모델 하이퍼 파라미터 4개 찾기

In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯
# [1] MobileNetV2 모델 
''''''
def build_model(hp):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=0.35,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None
      )
    base_model.trainable = True

    act_func = hp.Choice("act_func", ["relu", "selu", "elu", "softplus"])

    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=320,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act_func)(x)
    x = tf.keras.layers.Dense(units=128,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act_func)(x)
    x = tf.keras.layers.Dense(units=96,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act_func)(x)
    x = tf.keras.layers.Dense(units=32,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act_func)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = 3e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model


In [ ]:
# Keras_tuner 가 정상 작동하는지 확인
build_model(kt.HyperParameters())

ValueError: ignored

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=8,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
act_func (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'selu', 'elu', 'softplus'], 'ordered': False}


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

tuner.search(train_data_gen,validation_data=val_data_gen, epochs=7, callbacks=[keras.callbacks.TensorBoard("/tmp/tb_logs")])

# 텐서보드 출력
%load_ext tensorboard
%tensorboard --logdir /tmp/tb_logs

Trial 4 Complete [00h 42m 50s]
val_accuracy: 0.7362204790115356

Best val_accuracy So Far: 0.7532808184623718
Total elapsed time: 02h 52m 10s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
act_func          |relu              |relu              

Epoch 1/7
136/136 [==============================] - 371s 3s/step - loss: 0.4704 - accuracy: 0.8218 - val_loss: 0.7385 - val_accuracy: 0.6877
Epoch 2/7
136/136 [==============================] - 360s 3s/step - loss: 0.2693 - accuracy: 0.9052 - val_loss: 0.7782 - val_accuracy: 0.6955
Epoch 3/7
 29/136 [=====>........................] - ETA: 4:29 - loss: 0.2113 - accuracy: 0.9278

KeyboardInterrupt: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 320
units_1: 384
Score: 0.75
Trial summary
Hyperparameters:
num_layers: 2
units_0: 160
units_1: 64
Score: 0.75
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
units_1: 448
Score: 0.6666666865348816
Trial summary
Hyperparameters:
num_layers: 3
units_0: 160
units_1: 288
units_2: 64
Score: 0.6666666567325592


In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//batch_size,     # 한 프로퍼게이션때 한 배치가 쭉 통과하므로 그렇군
    epochs= 3,
    validation_data = val_data_gen,
    validation_steps = val_num//batch_size,
    callbacks=[my_callbacks]
)

# 쓰레기통

In [ ]:
''''''

''' [2] Xception 모델
def build_model(hp):
    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=None)

    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    base_model.trainable = True
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x=keras.layers.GlobalAveragePooling2D()(x)

    # l1 과 l2 를 모두 적용을 시키므로, 큰 값은 더욱 작게하기위해 노력을 하고, 또한 0 엣지를 가지게 하기 유리한 L1 을 모두 적용한다.
    l1_rate = hp.Float("l1_rate", min_value=1e-5, max_value=1e-1, sampling="log")
    l2_rate = hp.Float("l2_rate", min_value=1e-5, max_value=1e-1, sampling="log")

    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        # 정규화를 활성화 함수 이전에 배치하면 좋다고 수업시간에 한 학습을 바탕으로 모델링을 진행하였다.
        tf.keras.regularizers.l1_l2(l1=l1_rate, l2=l2_rate)
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=None, kernel_regularizer='l1_l2')(x)
        tf.keras.layers.BatchNormalization()(x),
        tf.keras.layers.Activation(activation=hp.Choice("activation", ["relu", "elu"]))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model
'''